<a href="https://colab.research.google.com/github/shreyasomkuwar26/multi_agent_investment_advisor/blob/main/Multi_Agent_Investment_advisor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comprehensive Investment Advisory Workflow Using CrewAI


In this notebook, we illustrate a sophisticated investment advisory system built using **CrewAI**. The workflow combines multiple AI agents, each equipped with specialized tools, to:
1. Gather financial data and news about a company.
2. Analyze the data comprehensively.
3. Provide actionable investment recommendations.

---
### Workflow Overview
1. Define custom tools to retrieve financial data, income statements, and stock prices.
2. Create agents for data gathering, analysis, and recommendation tasks.
3. Define tasks aligned with these agents' roles and objectives.
4. Configure a crew to execute these tasks sequentially, leveraging collaboration between agents.
5. Run the workflow and observe the results.

---
### Code Walkthrough
Below is the implementation.


### Step 1: Import Required Libraries and Declare in-built Tools

In [1]:
## initial setup for Colab.
## if you are using this in local jupyter notebook, skip this cell
!pip install crewai
!pip install crewai-tools
!pip install duckduckgo-search
!pip install yfinance
!pip install curl_cffi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.5/119.5 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 9.1 MB/s eta 0:

In [2]:
from crewai import Agent, Task
from crewai.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun
from datetime import datetime

# Current date for context
Now = datetime.now()
Today = Now.strftime("%d-%b-%Y")

# Define a web search tool
@tool("DuckDuckGo Search")
def search_tool(search_query: str):
    """Search the internet for information on a given topic"""
    return DuckDuckGoSearchRun().run(search_query)

### Step 2: Define Custom Tools

In [3]:
from crewai.tools import tool
import json

## Import Yahoo Finance APIs
import yfinance as yf
from curl_cffi import requests
import time

session = requests.Session(impersonate="chrome")

## A function is defined, that will work as a tool and that is provided to the framework (hence to agents) as a tool with the '@tool' decorator
## Note the function description  (purpose, usage) in doc strigs.
@tool ("Get current stock price")
def get_current_stock_price(symbol: str) -> str:
    """Use this function to get the current stock price for a given symbol.

    Args:
        symbol (str): The stock symbol.

    Returns:
        str: The current stock price or error message.
    """
    try:
        time.sleep(0.5)
        stock = yf.Ticker(symbol,session=session)

        current_price = stock.info.get("regularMarketPrice", stock.info.get("currentPrice"))
        return f"{current_price:.2f}" if current_price else f"Could not fetch current price for {symbol}"
    except Exception as e:
        return f"Error fetching current price for {symbol}: {e}"

@tool
def get_company_info(symbol: str):
    """Use this function to get company information and current financial snapshot for a given stock symbol.

    Args:
        symbol (str): The stock symbol.

    Returns:
        JSON containing company profile and current financial snapshot.
    """
    try:
        company_info_full = yf.Ticker(symbol,session=session).info
        if company_info_full is None:
            return f"Could not fetch company info for {symbol}"

        company_info_cleaned = {
            "Name": company_info_full.get("shortName"),
            "Symbol": company_info_full.get("symbol"),
            "Current Stock Price": f"{company_info_full.get('regularMarketPrice', company_info_full.get('currentPrice'))} {company_info_full.get('currency', 'USD')}",
            "Market Cap": f"{company_info_full.get('marketCap', company_info_full.get('enterpriseValue'))} {company_info_full.get('currency', 'USD')}",
            "Sector": company_info_full.get("sector"),
            "Industry": company_info_full.get("industry"),
            "City": company_info_full.get("city"),
            "Country": company_info_full.get("country"),
            "EPS": company_info_full.get("trailingEps"),
            "P/E Ratio": company_info_full.get("trailingPE"),
            "52 Week Low": company_info_full.get("fiftyTwoWeekLow"),
            "52 Week High": company_info_full.get("fiftyTwoWeekHigh"),
            "50 Day Average": company_info_full.get("fiftyDayAverage"),
            "200 Day Average": company_info_full.get("twoHundredDayAverage"),
            "Employees": company_info_full.get("fullTimeEmployees"),
            "Total Cash": company_info_full.get("totalCash"),
            "Free Cash flow": company_info_full.get("freeCashflow"),
            "Operating Cash flow": company_info_full.get("operatingCashflow"),
            "EBITDA": company_info_full.get("ebitda"),
            "Revenue Growth": company_info_full.get("revenueGrowth"),
            "Gross Margins": company_info_full.get("grossMargins"),
            "Ebitda Margins": company_info_full.get("ebitdaMargins"),
        }
        return json.dumps(company_info_cleaned)
    except Exception as e:
        return f"Error fetching company profile for {symbol}: {e}"

@tool
def get_income_statements(symbol: str):

    """Use this function to get income statements for a given stock symbol.

    Args:
    symbol (str): The stock symbol.

    Returns:
    JSON containing income statements or an empty dictionary.
    """
    try:
        stock = yf.Ticker(symbol,session=session)
        financials = stock.financials
        return financials.to_json(orient="index")
    except Exception as e:
        return f"Error fetching income statements for {symbol}: {e}"


### Step 3: Define the Agents



In [4]:
from crewai import Agent

# Agent for gathering company news and information
news_info_explorer = Agent(
    role='News and Info Researcher',
    goal='Gather and provide the latest news and information about a company from the internet',
    #llm='gpt-4o',
    llm='gpt-4.1-2025-04-14',
    verbose=True,
    backstory=(
        'You are an expert researcher, who can gather detailed information about a company. '
        'Consider you are on: ' + Today
    ),
    tools=[search_tool],
    cache=True,
    max_iter=5,
)

# Agent for gathering financial data
data_explorer = Agent(
    role='Data Researcher',
    goal='Gather and provide financial data and company information about a stock',
    #llm='gpt-4o',
    llm='gpt-4.1-2025-04-14',
    verbose=True,
    backstory=(
        'You are an expert researcher, who can gather detailed information about a company or stock. '
        'When using tools, use the stock symbol and add a suffix ".NS" to it. try with and without the suffix and see what works'
        'Consider you are on: ' + Today
    ),
    tools=[get_company_info, get_income_statements],
    cache=True,
    max_iter=5,
)

# Agent for analyzing data
analyst = Agent(
    role='Data Analyst',
    goal='Consolidate financial data, stock information, and provide a summary',
    #llm='gpt-4o',
    llm='gpt-4.1-2025-04-14',
    verbose=True,
    backstory=(
        'You are an expert in analyzing financial data, stock/company-related current information, and '
        'making a comprehensive analysis. Use Indian units for numbers (lakh, crore). '
        'Consider you are on: ' + Today
    ),
)

# Agent for financial recommendations
fin_expert = Agent(
    role='Financial Expert',
    goal='Considering financial analysis of a stock, make investment recommendations',
    #llm='gpt-4o',
    llm='gpt-4.1-2025-04-14',
    verbose=True,
    tools=[get_current_stock_price],
    max_iter=5,
    backstory=(
        'You are an expert financial advisor who can provide investment recommendations. '
        'Consider the financial analysis, current information about the company, current stock price, '
        'and make recommendations about whether to buy/hold/sell a stock along with reasons.'
        'When using tools, try with and without the suffix ".NS" to the stock symbol and see what works. '
        'Consider you are on: ' + Today
    ),
)

### Step 4: Define the Tasks

In [5]:
from crewai import Task

# Task to gather financial data of a stock
get_company_financials = Task(
    description="Get financial data like income statements and other fundamental ratios for stock: {stock}",
    expected_output="Detailed information from income statement, key ratios for {stock}. "
                    "Indicate also about current financial status and trend over the period.",
    agent=data_explorer,
)

# Task to gather company news
get_company_news = Task(
    description="Get latest news and business information about company: {stock}",
    expected_output="Latest news and business information about the company. Provide a summary also.",
    agent=news_info_explorer,
)

# Task to analyze financial data and news
analyse = Task(
    description="Make thorough analysis based on given financial data and latest news of a stock",
    expected_output="Comprehensive analysis of a stock outlining financial health, stock valuation, risks, and news. "
                    "Mention currency information and number units in Indian context (lakh/crore).",
    agent=analyst,
    context=[get_company_financials, get_company_news],
    output_file='Analysis.md',
)

# Task to provide financial advice
advise = Task(
    description="Make a recommendation about investing in a stock, based on analysis provided and current stock price. "
                "Explain the reasons.",
    expected_output="Recommendation (Buy / Hold / Sell) of a stock backed with reasons elaborated."
                    "Response in Mark down format.",
    agent=fin_expert,
    context=[analyse],
    output_file='Recommendation.md',
)

### Step 5: Set Up the Crew

In [6]:
from crewai import Crew, Process
from datetime import datetime

# Callback function to print a timestamp
def timestamp(Input):
    print(datetime.now())

# Define the crew with agents and tasks in sequential process
crew = Crew(
    agents=[data_explorer, news_info_explorer, analyst, fin_expert],
    tasks=[get_company_financials, get_company_news, analyse, advise],
    verbose=True,
    Process=Process.sequential,
    step_callback=timestamp,
)

### Step 5: Run the Crew and Observe Results

In [8]:
# Set your OpenAI API key or any other LLM API key
from google.colab import userdata
import os

# Set your Groq API key or any other LLM API key
os.environ['OPENAI_API_KEY'] = userdata.get('openai_key')

# Run the crew with a specific stock
result = crew.kickoff(inputs={'stock': 'RELIANCE'})

# Print the final result
print("Final Result:", result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 74a8cc91-cfb3-4b8d-9cde-f312ab941672                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Researcher                                                                                         │
│                                                                                                                 │
│  Task: Get financial data like income statements and other fundamental ratios for stock: RELIANCE               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

2025-07-19 21:23:10.175395

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Researcher                                                                                         │
│                                                                                                                 │
│  Thought: Thought: To provide a detailed answer, I need the latest company financial snapshot and income        │
│  statement data for RELIANCE. I'll begin by retrieving the company information, which typically includes key    │
│  ratios and recent financials.                                                                                  │
│                                                                                                                 │
│  Using Tool: get_company_info                                                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7a4f0fed5a20>


╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Financial Expert                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ## Investment Recommendation for Reliance Industries Ltd (RELIANCE.NS) as of 19-Jul-2025                       │
│                                                                                                                 │
│  **Recommendation:** **HOLD** (with a positive long-term bias)                                                  │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### Reasoning                                                                                                  │
│                                                                                                                 │
│  #### 1. **Current Valuation**                                                                                  │
│                                                                                                                 │
│  - **Current Price:** ₹1,476.00 per share (as confirmed).                                                       │
│  - **Market Context:** The stock has rallied approximately 24% YTD and trades at a premium P/E (24.51),         │
│  reflective of optimistic future growth already "priced in".                                                    │
│  - **Risks of Overvaluation:** With the recent rally, there's a meaningful risk that near-term upside is        │
│  capped if quarterly results or new investments disappoint.                                                     │
│                                                                                                                 │
│  #### 2. **Financial Fundamentals**                                                                             │
│                                                                                                                 │
│  - **Revenue & Profit Growth:** Impressive CAGR of ~12% in revenues and continuous growth in net profit,        │
│  underlining operational execution.                                                                             │
│  - **Cash Reserves & Liquidity:** ₹2,25,211 crore in cash and strong operating cash flows strengthen its        │
│  balance sheet resilience—key for a conglomerate deepening its capex commitments.                               │
│  - **Negative Free Cash Flow:** The most notable concern; ongoing heavy investments have led to negative free   │
│  cash flow (-₹4,658 crore latest). This is manageable given current liquidity, but continued negative FCF       │
│  needs periodic review to ensure investments start yielding.                                                    │
│                                                                                                                 │
│  #### 3. **Operational Strengths**                                                                              │
│                                                                                                                 │
│  - **Diversification:** RIL’s transformation into an energy–retail–digital giant insulates it from shocks in    │
│  any single sector.                                    

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 9f675c0f-1e25-49d0-829b-38a75f26c3f5                                                                     │
│  Agent: Financial Expert                                                                                        │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 74a8cc91-cfb3-4b8d-9cde-f312ab941672                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ## Investment Recommendation for Reliance Industries Ltd (RELIANCE.NS) as of 19-Jul-2025         │
│                                                                                                                 │
│  **Recommendation:** **HOLD** (with a positive long-term bias)                                                  │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### Reasoning                                                                                                  │
│                                                                                                                 │
│  #### 1. **Current Valuation**                                                                                  │
│                                                                                                                 │
│  - **Current Price:** ₹1,476.00 per share (as confirmed).                                                       │
│  - **Market Context:** The stock has rallied approximately 24% YTD and trades at a premium P/E (24.51),         │
│  reflective of optimistic future growth already "priced in".                                                    │
│  - **Risks of Overvaluation:** With the recent rally, there's a meaningful risk that near-term upside is        │
│  capped if quarterly results or new investments disappoint.                                                     │
│                                                                                                                 │
│  #### 2. **Financial Fundamentals**                                                                             │
│                                                                                                                 │
│  - **Revenue & Profit Growth:** Impressive CAGR of ~12% in revenues and continuous growth in net profit,        │
│  underlining operational execution.                                                                             │
│  - **Cash Reserves & Liquidity:** ₹2,25,211 crore in cash and strong operating cash flows strengthen its        │
│  balance sheet resilience—key for a conglomerate deepening its capex commitments.                               │
│  - **Negative Free Cash Flow:** The most notable concern; ongoing heavy investments have led to negative free   │
│  cash flow (-₹4,658 crore latest). This is manageable given current liquidity, but continued negative FCF       │
│  needs periodic review to ensure investments start yielding.                                                    │
│                                                                                                                 │
│  #### 3. **Operational Strengths**                                                                              │
│                                                                                                                 │
│  - **Diversification:** RIL’s transformation into an e

Final Result: ## Investment Recommendation for Reliance Industries Ltd (RELIANCE.NS) as of 19-Jul-2025

**Recommendation:** **HOLD** (with a positive long-term bias)

---

### Reasoning

#### 1. **Current Valuation**

- **Current Price:** ₹1,476.00 per share (as confirmed).
- **Market Context:** The stock has rallied approximately 24% YTD and trades at a premium P/E (24.51), reflective of optimistic future growth already "priced in".
- **Risks of Overvaluation:** With the recent rally, there's a meaningful risk that near-term upside is capped if quarterly results or new investments disappoint.

#### 2. **Financial Fundamentals**

- **Revenue & Profit Growth:** Impressive CAGR of ~12% in revenues and continuous growth in net profit, underlining operational execution.
- **Cash Reserves & Liquidity:** ₹2,25,211 crore in cash and strong operating cash flows strengthen its balance sheet resilience—key for a conglomerate deepening its capex commitments.
- **Negative Free Cash Flow:** The mos


---
### Conclusion
This workflow demonstrates a detailed investment advisory process using multiple agents and tasks. The system showcases the integration of custom tools and collaborative agents in CrewAI to provide actionable financial insights.
